In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv("customers-1000.csv")#df is the data frame containing the whole content of file
columns = np.array(df.columns)#to know the number of colums in dataframe(csv) file
noOfColumns = len(columns)
noOfRows = len(df.index)
print(f"Total columns = {noOfColumns}")
print(f"Total rows = {noOfRows}")

print([df.iloc[:,1].duplicated()])
# clusteredColumns = np.array([])
# for i in noOfColumns:
#     if i.duplicate()
    

class dataFrame:
    mainContent = df
    def bestMatch():
        print(mainContent.iloc[:,1])

Total columns = 12
Total rows = 1000
[0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: Customer Id, Length: 1000, dtype: bool]
